In [1]:
from typing import List, Optional
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import re

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Андрей\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Андрей\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Андрей\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
train = pd.read_csv('./data/train_tweets.csv', index_col=0)
test = pd.read_csv('./data/test_tweets.csv',index_col=0)

In [3]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [4]:
train

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
3,0,bihday your majesty
4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
5,0,factsguide: society now #motivation
...,...,...
31958,0,ate @user isz that youuu?ðððððððððâ¤ï¸
31959,0,to see nina turner on the airwaves trying to wrap herself in the mantle of a genuine hero like shirley chisolm. #shame #imwithher
31960,0,listening to sad songs on a monday morning otw to work is sad


### Для работы объединим train_df и test_df.

In [5]:
df = pd.concat([train, test], axis=0)

In [6]:
df

,label,tweet
id,,
1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
3,0.0,bihday your majesty
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
5,0.0,factsguide: society now #motivation
...,...,...
49155,NaN,thought factory: left-right polarisation! #trump #uselections2016 #leadership #politics #brexit #blm &gt;3
49156,NaN,feeling like a mermaid ð #hairflip #neverready #formal #wedding #gown #dresses #mermaid â¦
49157,NaN,"#hillary #campaigned today in #ohio((omg)) &amp; used words like ""assets&amp;liability"" never once did #clinton say thee(word) #radicalization"


### 1. Удалим @user из всех твитов с помощью паттерна "@[\w]*".


In [7]:
def user_delete(df: pd.DataFrame) -> pd.DataFrame :
    """
    функция которая удаляет @user

    -------------------------------------
     Returns
     pd.dataframe

    """

    df['tweet'] = df.tweet.str.replace('(@[\w]*)' , '')
    
    return df

In [8]:
user_delete(df)

<ipython-input-7-4286dee9b84e>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tweet'] = df.tweet.str.replace('(@[\w]*)' , '')


,label,tweet
id,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
3,0.0,bihday your majesty
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
5,0.0,factsguide: society now #motivation
...,...,...
49155,NaN,thought factory: left-right polarisation! #trump #uselections2016 #leadership #politics #brexit #blm &gt;3
49156,NaN,feeling like a mermaid ð #hairflip #neverready #formal #wedding #gown #dresses #mermaid â¦
49157,NaN,"#hillary #campaigned today in #ohio((omg)) &amp; used words like ""assets&amp;liability"" never once did #clinton say thee(word) #radicalization"


2. Изменим регистр твитов на нижний с помощью .lower().

In [9]:
df['tweet'] = df.tweet.str.lower()
df.head()

,label,tweet
id,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
3,0.0,bihday your majesty
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
5,0.0,factsguide: society now #motivation


3. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [10]:
def replace_dict(text, my_dict):
    
    text_new = str()
    for word in text.split():
        if word in my_dict.keys():
            word = my_dict.get(word)
        text_new = text_new + word+ ' '
        
    return text_new

In [11]:
vecfoo = np.vectorize(replace_dict)
df['tweet_apostrophe'] = vecfoo(df['tweet'], apostrophe_dict)
df.head(20)

,label,tweet,tweet_apostrophe
id,,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked
3,0.0,bihday your majesty,bihday your majesty
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
5,0.0,factsguide: society now #motivation,factsguide: society now #motivation
6,0.0,[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo,[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo
7,0.0,camping tomorrow dannyâ¦,camping tomorrow dannyâ¦
8,0.0,the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams #hate #imagine #actorslife #revolutionschool #girl,the next school year is the year for exams.ð¯ cannot think about that ð­ #school #exams #hate #imagine #actorslife #revolutionschool #girl
9,0.0,we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers â¦,we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers â¦


### 4 . Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [12]:
vecfoo = np.vectorize(replace_dict)
df['tweet_short_word'] = vecfoo(df['tweet'], short_word_dict)
df.head()

,label,tweet,tweet_apostrophe,tweet_short_word
id,,,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
3,0.0,bihday your majesty,bihday your majesty,bihday your majesty
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦
5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation


### 5. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [13]:
vecfoo = np.vectorize(replace_dict)
df['tweet_emoticon'] = vecfoo(df['tweet_short_word'], emoticon_dict)
df.head()

,label,tweet,tweet_apostrophe,tweet_short_word,tweet_emoticon
id,,,,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
3,0.0,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦
5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation


### 6. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

In [14]:
df['tweet_punctuation'] = [re.sub(r'[^\w\s]', ' ', x) for x in df['tweet_emoticon']]
df.head()

,label,tweet,tweet_apostrophe,tweet_short_word,tweet_emoticon,tweet_punctuation
id,,,,,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked
3,0.0,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,model i love you take with you all the time in urð ð ð ð ð ð ð ð
5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide society now motivation


### 7. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [15]:
df['tweet_special_characters'] = [re.sub(r'[^a-zA-Z0-9]', ' ', x) for x in df['tweet_punctuation']]
df.head()

,label,tweet,tweet_apostrophe,tweet_short_word,tweet_emoticon,tweet_punctuation,tweet_special_characters
id,,,,,,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked
3,0.0,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,model i love you take with you all the time in urð ð ð ð ð ð ð ð,model i love you take with you all the time in ur
5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide society now motivation,factsguide society now motivation


### 8. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'


In [16]:
df['tweet_numbers_for_spaces'] = [re.sub(r'[^a-zA-Z]', ' ', x) for x in df['tweet_special_characters']]
df.head()

,label,tweet,tweet_apostrophe,tweet_short_word,tweet_emoticon,tweet_punctuation,tweet_special_characters,tweet_numbers_for_spaces
id,,,,,,,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked
3,0.0,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,model i love you take with you all the time in urð ð ð ð ð ð ð ð,model i love you take with you all the time in ur,model i love you take with you all the time in ur
5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide society now motivation,factsguide society now motivation,factsguide society now motivation


### 9. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [17]:
def drop_one(text):
    return ' '.join([w for w in text.split() if len(w)>1])

In [18]:
vecfoo = np.vectorize(drop_one)
df['tweet_droped_one_simbol_words'] = vecfoo(df['tweet_numbers_for_spaces'])
df.head()

,label,tweet,tweet_apostrophe,tweet_short_word,tweet_emoticon,tweet_punctuation,tweet_special_characters,tweet_numbers_for_spaces,tweet_droped_one_simbol_words
id,,,,,,,,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit can use cause they don offer wheelchair vans in pdx disapointed getthanked
3,0.0,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,model i love you take with you all the time in urð ð ð ð ð ð ð ð,model i love you take with you all the time in ur,model i love you take with you all the time in ur,model love you take with you all the time in ur
5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide society now motivation,factsguide society now motivation,factsguide society now motivation,factsguide society now motivation


### 10. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [19]:
df['tweet_tokenize'] = [nltk.tokenize.word_tokenize(w) for w in df['tweet_droped_one_simbol_words']]
df.head()


,label,tweet,tweet_apostrophe,tweet_short_word,tweet_emoticon,tweet_punctuation,tweet_special_characters,tweet_numbers_for_spaces,tweet_droped_one_simbol_words,tweet_tokenize
id,,,,,,,,,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]"
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit can use cause they don offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, use, cause, they, don, offer, wheelchair, vans, in, pdx, disapointed, getthanked]"
3,0.0,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,"[bihday, your, majesty]"
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,model i love you take with you all the time in urð ð ð ð ð ð ð ð,model i love you take with you all the time in ur,model i love you take with you all the time in ur,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, time, in, ur]"
5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide society now motivation,factsguide society now motivation,factsguide society now motivation,factsguide society now motivation,"[factsguide, society, now, motivation]"


### 11. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [20]:
def drop_stop(text):
    
    stop_words = set(nltk.corpus.stopwords.words("english"))
    
    return [w for w in text if not w in stop_words]

In [21]:
df['tweet_token_filtered'] = list(map(drop_stop, df['tweet_tokenize']))
df.head()

,label,tweet,tweet_apostrophe,tweet_short_word,tweet_emoticon,tweet_punctuation,tweet_special_characters,tweet_numbers_for_spaces,tweet_droped_one_simbol_words,tweet_tokenize,tweet_token_filtered
id,,,,,,,,,,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]"
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit can use cause they don offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, use, cause, they, don, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]"
3,0.0,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]"
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,model i love you take with you all the time in urð ð ð ð ð ð ð ð,model i love you take with you all the time in ur,model i love you take with you all the time in ur,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, time, in, ur]","[model, love, take, time, ur]"
5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide society now motivation,factsguide society now motivation,factsguide society now motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]"


### 12. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [22]:
def stemmer(text):
    
    stemmer = nltk.stem.PorterStemmer()
    
    text_new = []
    for word in text:
        word = stemmer.stem(word)
        text_new.append(word)
        
    return text_new

In [23]:
df['tweet_stemmed'] = list(map(stemmer, df['tweet_token_filtered']))
df.head()

,label,tweet,tweet_apostrophe,tweet_short_word,tweet_emoticon,tweet_punctuation,tweet_special_characters,tweet_numbers_for_spaces,tweet_droped_one_simbol_words,tweet_tokenize,tweet_token_filtered,tweet_stemmed
id,,,,,,,,,,,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[father, dysfunct, selfish, drag, kid, dysfunct, run]"
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit can use cause they don offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, use, cause, they, don, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]","[thank, lyft, credit, use, caus, offer, wheelchair, van, pdx, disapoint, getthank]"
3,0.0,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]"
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,model i love you take with you all the time in urð ð ð ð ð ð ð ð,model i love you take with you all the time in ur,model i love you take with you all the time in ur,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, time, in, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide society now motivation,factsguide society now motivation,factsguide society now motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]"


### 13. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [26]:
def lemmatizer(text):
    
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    
    text_new = []
    for word in text:
        word = lemmatizer.lemmatize(word)
        text_new.append(word)
        
    return text_new

In [27]:
df['tweet_lemmatized'] = list(map(lemmatizer, df['tweet_token_filtered']))
df.head()

,label,tweet,tweet_apostrophe,tweet_short_word,tweet_emoticon,tweet_punctuation,tweet_special_characters,tweet_numbers_for_spaces,tweet_droped_one_simbol_words,tweet_tokenize,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
id,,,,,,,,,,,,,
1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[father, dysfunct, selfish, drag, kid, dysfunct, run]","[father, dysfunctional, selfish, drag, kid, dysfunction, run]"
2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit i can t use cause they don t offer wheelchair vans in pdx disapointed getthanked,thanks for lyft credit can use cause they don offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, use, cause, they, don, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]","[thank, lyft, credit, use, caus, offer, wheelchair, van, pdx, disapoint, getthank]","[thanks, lyft, credit, use, cause, offer, wheelchair, van, pdx, disapointed, getthanked]"
3,0.0,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦,model i love you take with you all the time in urð ð ð ð ð ð ð ð,model i love you take with you all the time in ur,model i love you take with you all the time in ur,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, time, in, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
5,0.0,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide: society now #motivation,factsguide society now motivation,factsguide society now motivation,factsguide society now motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


### 14. Сохраним результат предобработки в pickle-файл.

In [28]:
df.to_pickle('data/clean_tweets.pkl')